In [8]:
import json
import os
from py2neo import Graph, Node, Relationship

In [9]:
profile_dir = r"out_profile"
post_dir = r"out_post"

files = ["acm", "antariksh", "arya", "astra", "codestars", "compute", "djscsi", "ecell", "gdsc", "iete", "init", "isaca", "isme", "karting", "miles", "nova", "panache", "ras", "robocon", "s4ds", "sigai", "speedsters", "synapse", "unicode"]

In [10]:
from dotenv import load_dotenv

load_dotenv()

URI = os.environ.get("NEO4J_URI")
USERNAME = os.environ.get("NEO4J_USERNAME")
PASSWORD = os.environ.get("NEO4J_PASSWORD")

In [11]:
graph = Graph(URI, auth=(USERNAME, PASSWORD))

In [12]:
com_acc_ids = []

for file in files:
    profilefile = os.path.join(profile_dir, file)
    profilefile += '.json'
    
    with open(profilefile, 'r', encoding='utf-8') as f:
        profile_data = json.load(f)

    com_acc_ids.append(profile_data["id"])

In [13]:
len(com_acc_ids)

24

In [15]:
# create all the committee nodes
for file in files:
    profilefile = os.path.join(profile_dir, file)
    profilefile += '.json'
    
    with open(profilefile, 'r', encoding='utf-8') as f:
        profile_data = json.load(f)
    
    # Create and add the committee node
    committee = Node("Committee",
                     id=profile_data['id'],
                     username=profile_data['username'],
                     full_name=profile_data['full_name'],
                     bio=profile_data['bio'],
                     num_followers=profile_data['num_followers'],
                     num_following=profile_data['num_following'],
                     highlight_reel_count=profile_data['highlight_rell_count'],
                     num_posts=profile_data['num_posts'],
                     category=profile_data['category'])
    
    graph.create(committee)
    
print("All committee nodes have been created.")

All committee nodes have been created.


In [16]:
# add followers and posts
for file in files:
    profilefile = os.path.join(profile_dir, file)
    profilefile += '.json'
    
    with open(profilefile, 'r', encoding='utf-8') as f:
        profile_data = json.load(f)

    postfile = os.path.join(post_dir, file)
    postfile += '.json'
    
    with open(postfile, 'r', encoding='utf-8') as f:
        posts_data = json.load(f)
    
    committee = graph.nodes.match("Committee", id=profile_data['id']).first()
    
    # Create Follower relationships or link to other committee nodes
    for follower in profile_data['followers']:
        if follower['id'] in com_acc_ids:
            # Follower is a committee, create a relationship
            follower_node = graph.nodes.match("Committee", id=follower['id']).first()
        else:
            # Follower is not a committee, create a Follower node
            follower_node = Node("Follower", id=follower['id'], username=follower['username'])
            graph.create(follower_node)
        
        follows_rel = Relationship(follower_node, "FOLLOWS", committee)
        graph.create(follows_rel)
    
    # Create Post nodes and relationships
    for post in posts_data:
        post_attributes = {
            'id': post['id'],
            'type': post['type'],
            'comments': post['comments'],
            'likes': post['likes'],
            'time_taken': post['time_taken']
        }
        
        # Add caption if it exists
        if 'caption' in post and post['caption']:
            post_attributes['caption'] = post['caption']
        
        # Add type-specific attributes
        if post['type'] == 'GraphSidecar':
            post_attributes['num_children'] = post.get('num_children', 0)
        elif post['type'] == 'GraphVideo':
            post_attributes['views'] = post.get('views', 0)
        
        post_node = Node("Post", **post_attributes)
        posted_rel = Relationship(committee, "POSTED", post_node)
        graph.create(post_node)
        graph.create(posted_rel)
    
    print(f"Followers and posts for {profile_data['username']} have been successfully added.")


Followers and posts for djsanghvi_acm have been successfully added.
Followers and posts for djs_antariksh have been successfully added.
Followers and posts for djs_arya have been successfully added.
Followers and posts for djs.astra have been successfully added.
Followers and posts for djsce_codestars have been successfully added.
Followers and posts for djscompute have been successfully added.
Followers and posts for djcsi have been successfully added.
Followers and posts for ecell_djsce have been successfully added.
Followers and posts for gdsc.djsce have been successfully added.
Followers and posts for djsce_iete have been successfully added.
Followers and posts for djinit.ai have been successfully added.
Followers and posts for djsisaca have been successfully added.
Followers and posts for djsce_isme have been successfully added.
Followers and posts for djskartingindia have been successfully added.
Followers and posts for djs_miles have been successfully added.
Followers and posts 

In [17]:
print("this is taking long")

this is taking long
